<a href="https://colab.research.google.com/github/ThisGuy245/AI-Tasks/blob/main/AI_Task_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import time

# Load the model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def generate_response(prompt, max_new_tokens=100, temperature=0.7, top_k=50):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    start_time = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        no_repeat_ngram_size=2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    end_time = time.time()
    response_time = end_time - start_time
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    ai_response = response.split("AI:")[-1].strip()
    return ai_response, response_time


def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    return torch.exp(loss)

# Main conversation loop
context = "You are an AI assistant specialized in discussing AI advancements. Provide informative and coherent responses about AI progress, its impact on society, and potential future developments."
max_context_length = 1000
conversation_history = ""

print("Welcome to the AI Chat! Let's discuss AI advancements.")
print("You can start chatting. Type 'exit' to end the conversation.")

while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        break

    prompt = f"{context}\n{conversation_history}\nUser: {user_input}\nAI:"
    ai_response, response_time = generate_response(prompt, max_new_tokens=100, temperature=0.8, top_k=40)

    print(f"AI: {ai_response}")
    print(f"Response time: {response_time:.2f} seconds")

    conversation_history += f"\nUser: {user_input}\nAI: {ai_response}"
    if len(conversation_history) > max_context_length:
        conversation_history = conversation_history[-max_context_length:]

    perplexity = calculate_perplexity(model, tokenizer, f"User: {user_input}\nAI: {ai_response}")
    print(f"Perplexity of last exchange: {perplexity.item():.2f}")


print("Thank you for chatting! Here's a summary of our conversation:")
print(conversation_history)

# Final evaluation
final_perplexity = calculate_perplexity(model, tokenizer, conversation_history)
print(f"\nOverall conversation perplexity: {final_perplexity.item():.2f}")
